# Bench Buddy Simulation

**Example title** 
- example bullet
- example bullet

Let's start by actually doing something!

# Set up environment

In [ ]:
# Optional installs for packages

# pip3 install numpy pandas matplotlib seaborn scikit-learn scipy jupyterlab
# pip3 install tensorflow torch torchvision
# pip3 install pydrake

In [ ]:
import os
import time
from pathlib import Path
from textwrap import dedent

import numpy as np
from pydrake.all import (
    Context,
    DiagramBuilder,
    InverseKinematics,
    MultibodyPlant,
    PiecewisePolynomial,
    RigidTransform,
    RotationMatrix,
    Simulator,
    Solve,
    StartMeshcat,
    TrajectorySource,
)

from manipulation import running_as_notebook
from manipulation.exercises.trajectories.rrt_planner.robot import (
    ConfigurationSpace,
    Range,
)
from manipulation.exercises.trajectories.rrt_planner.rrt_planning import (
    RRT,
    TreeNode,
)
from manipulation.letter_generation import create_sdf_asset_from_letter
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.station import LoadScenario, MakeHardwareStation

In [ ]:
meshcat = StartMeshcat()
print(f"Meshcat URL: {meshcat.web_url()}")
print("Click the link above to open Meshcat in your browser!")

# Set up the scene

In [ ]:
outdir = "scenarios"
assets_dir = "assets"
os.makedirs(outdir, exist_ok=True)
os.makedirs(assets_dir, exist_ok=True)

# Set up necessary controllers & ??

Setting up our PD controller with our iiwa robot.

In [ ]:
def create_IIWA14_diagram_with_controller(
    Kp: float,
    Kd: float,
    q_desired: np.ndarray,
):
    """Create a diagram with the IIWA14 and PD controller (gravity compensated)."""

    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
    parser = Parser(plant, scene_graph)

    # Load the IIWA model
    iiwa = parser.AddModelsFromUrl(
        "package://drake_models/iiwa_description/urdf/iiwa14_primitive_collision.urdf"
    )[0]

    # Weld base frame to world
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("iiwa_link_0"))
    plant.Finalize()

    # Create inverse dynamics PD controller (adds gravity compensation automatically)
    num_joints = plant.num_positions(iiwa)
    controller = builder.AddSystem(
        InverseDynamicsController(
            plant=plant,
            Kp=[Kp] * num_joints,
            Ki=[0] * num_joints,
            Kd=[Kd] * num_joints,
            has_reference_acceleration=False,
        )
    )

    # Constant source for desired joint positions
    desired_source = builder.AddSystem(ConstantVectorSource(q_desired))

    # Connect the controller
    builder.Connect(plant.get_state_output_port(iiwa), controller.get_input_port_estimated_state())
    builder.Connect(desired_source.get_output_port(), controller.get_input_port_desired_state())
    builder.Connect(controller.get_output_port_control(), plant.get_actuation_input_port(iiwa))

    # Add data logger
    logger = builder.AddSystem(VectorLogSink(plant.num_multibody_states()))
    builder.Connect(plant.get_state_output_port(iiwa), logger.get_input_port())

    # Add Meshcat visualizer
    MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

    diagram = builder.Build()
    return diagram, plant, iiwa, logger


In [ ]:
def simulate_IIWA14_with_controller(
    q0: np.ndarray,
    controller_gain: float,
    q_desired: np.ndarray,
    simulation_time=3.0,
    set_target_realtime_rate=True,
) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """Simulate IIWA14 with PD controller"""

    diagram, plant, iiwa, logger = create_IIWA14_diagram_with_controller(
        controller_gain, q_desired
    )

    diagram_context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyMutableContextFromRoot(diagram_context)
    plant.SetPositions(plant_context, q0)

    simulator = Simulator(diagram, diagram_context)
    if set_target_realtime_rate:
        simulator.set_target_realtime_rate(1.0)

    simulator.AdvanceTo(simulation_time)

    # Extract logged data
    log = logger.FindLog(diagram_context)
    times = log.sample_times()
    states = log.data()

    final_positions = plant.GetPositions(plant_context)
    return final_positions, times, states

# Simulate bench

# Simulate body, neck, head, & arms

# Simulate bar & its behavior

In [ ]:
# Need range for fastest and slowest that the bar could drop

In [ ]:
# Need range for farthest and closest that the bar could follow

In [ ]:
# Within those two ranges need to randomize trajectory of the bar lowering

# Test cases & saving test results

# Analyzing test results